# Quantum Simulator: Service Allocation

This solution identifies a set of optimal trajectories that complete all the jobs on a given widget while minimizing the congestion across all resources. It helps in reducing operational cost and turnaround time in production processes. The solution utilizes quantum computing simulator for optimization, making it scalable and robust.
### Prerequisite

The kernel comes pre-installed with the required packages. Else ensure to have the following Python Packages in your environment at minimum:

    - scipy
    - numpy
    - pandas
    - qubovert

 ### Contents

1. [Input Data](#Input-Format)
1. [Creating Model](#Creating-Model)
1. [Batch Transform](#Batch-Transform)
1. [Output](#Output)
1. [Real Time Transform](#Invoking-through-Endpoint)

### Input Format
* The input should be a zipped file which contains two csv. Name of all the csv files should match with the name of csv files in sample data which are as follows
* resource_data.csv : This file will contain columns like path, machin_id and all the resources columns which are in the system, all these resource columns provides information for flow details for each available path, ensure that names of the column should be same and exactly in the same order as in sample data set, cloumns of resource details can change according to the available number of resources in the system.
* resources_constraint.csv : This file will contain columns like Path, and different columns for each machine id, this file provides information of all the available paths for each machine, see the sample data for format.

<b> Note: 
    Input file from sage_maker should be of the form csv which should be zipped .<br>Ensure Content-Type is 'application/zip'
</b>

### Input instructions
 
• Supported content types: ` application/zip`

•  Input file should be of csv type and zipped, also all the csv files should be in the format mentioned in Input Format section

### Output interpretation


• Output file will contain the result in dictionary format

• Each key-value pair will give information of path to follow for each machine


## Importing libraries for runtime

In [1]:
import pandas as pd
import boto3
import re

## Creating Model


In [2]:
model_package_arn = 'put your arn here'

In [3]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [4]:
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


## Batch Transform

Now that model is ready, we can deploy the model and obtain solutions.

The output will generate results in dictionary format.


In [ ]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.xlarge')
transformer.transform('put s3 bucket path of input file', content_type='application/zip')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")
bucketFolder = transformer.output_path.rsplit('/')[3]

In [6]:
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
bucket_name="put your bucket name here""
with open('optimum_route.json', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/Input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


## Output

The processed output is of dictionary format giving information like which path to follow for each package
    

In [7]:
f = open('optimum_route.json', 'r')
file_contents = f.read()
print (file_contents)
f.close()

{"x(0)": "path one for machine_ 1", "x(3)": "path four for machine_ 2", "x(6)": "path seven for machine_ 3"}


## Invoking through Endpoint

In [8]:
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage

from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit

role = get_execution_role()

sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [9]:
content_type='application/zip'
model_name='process-optimizer'
real_time_inference_instance_type='ml.m5.xlarge'

In [10]:
model_package_arn = 'put your arn here'

In [11]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [12]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [13]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

### Invoking endpoint result through CLI command

In [14]:
file_name="Input.zip"

In [15]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'application/zip' --region us-east-2 output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


In [16]:
f = open('output.json', 'r')
file_contents = f.read()
print (file_contents)
f.close()

{"x(0)": "path one for machine_ 1", "x(3)": "path four for machine_ 2", "x(6)": "path seven for machine_ 3"}


In [18]:
predictor.delete_endpoint

<bound method RealTimePredictor.delete_endpoint of <sagemaker.predictor.RealTimePredictor object at 0x7f5d4f428390>>